lungeErrorDetection

In [19]:
TRAIN_PATH = ''
TEST_PATH = ''
labels_ = []

In [20]:
import mediapipe as mp
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_csv(TRAIN_PATH)
df.loc[df["label"] == labels_[0], "label"] = 0
df.loc[df["label"] == labels_[1], "label"] = 1

In [22]:
with open("./models/lunge_input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [23]:
X = df.drop("label", axis=1)
y = df["label"].astype("int")

X = pd.DataFrame(sc.transform(X))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train AI

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rf = RandomForestClassifier(n_estimators=120,
                          max_depth=30,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)
rf.fit(X_train, y_train)
model_results = rf.predict(X_test)

In [27]:
p_score = precision_score(y_test, model_results, average=None, labels=[1, 0])
a_score = accuracy_score(y_test, model_results)
r_score = recall_score(y_test, model_results, average=None, labels=[1, 0])
f1_score_result = f1_score(y_test, model_results, average=None, labels=[1, 0])
cm = confusion_matrix(y_test, model_results, labels=[1, 0])

eval AI

In [28]:
test_df = pd.read_csv(TEST_PATH)
test_df = test_df.sample(frac=1).reset_index(drop=True)

# Categorizing label
test_df.loc[test_df["label"] == labels_[0], "label"] = 0
test_df.loc[test_df["label"] == labels_[1], "label"] = 1

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"].astype("int")

test_x = pd.DataFrame(sc.transform(test_x))

model_results = rf.predict(test_x)

In [29]:
p_score = precision_score(test_y, model_results, average=None, labels=[1, 0])
a_score = accuracy_score(test_y, model_results)
r_score = recall_score(test_y, model_results, average=None, labels=[1, 0])
f1_score_result = f1_score(test_y, model_results, average=None, labels=[1, 0])
cm = confusion_matrix(test_y, model_results, labels=[1, 0])

In [34]:
print('RandomForest')
print(f'P_Score is {p_score}')
print(f'Accuracy_Score is {a_score}')
print(f'Recall_Score is {r_score}')
print(f'F1_Score is {f1_score_result}')

RandomForest
P_Score is [0.95463511 0.89666667]
Accuracy_Score is 0.9232158988256549
Recall_Score is [0.88644689 0.95900178]
F1_Score is [0.91927825 0.92678725]


dump model

In [33]:
with open("./models/lunge_errorDetection.pkl", "wb") as f:
    pickle.dump(rf, f)